In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [21]:
log_dir1 = 'log/20210222_111136_mcmc/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [22]:
log_dir2 = 'log/20210202_195936_hmc/'
exp_data2 = load_obj( 'complete_data', log_dir2)
ps_hmc = load_obj('hmc_post_samples', log_dir2)


In [23]:
log_dir3 = 'log/20210225_155650_db_2steps//'
exp_data = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)
# particles.particles[5].particles['beta']

99


In [24]:
exp_data.raw_data['random_seed']

2

In [25]:
exp_data1.raw_data['betabeta'] = exp_data1.raw_data['beta'].reshape(6,1)@exp_data1.raw_data['beta'].reshape(6,1).T

In [26]:
def get_betabeta(betas):
    N = betas.shape[0]
    J = betas.shape[1]
    betabeta = np.empty((N, J, J))
    for i in range(N):
        betabeta[i] = betas[i]@betas[i].T
    return betabeta


## Plot weighted histograms

In [27]:
particles = load_obj('particles', log_dir3)
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
# ps_smc2['betabeta'] = get_betabeta(ps_smc2['beta'])

In [28]:
def get_histogram_data(ps_smc2, name):
    if name == 'alpha':
        plot_data = []
        for col in range(6):
            weights =  exp_and_normalise(particles.weights)
            hist, bin_edges = np.histogram(ps_smc2[name][:,col], weights = weights)
            center_bin_edges = bin_edges[1:] + ((bin_edges[1:] - bin_edges[:-1])/2.)
            plot_data_col = pd.DataFrame(hist, columns=['value'])
            plot_data_col['bin'] = center_bin_edges
            plot_data_col['col'] = col
            plot_data_col['row'] = 1
            plot_data.append(plot_data_col)
        return pd.concat(plot_data)
    if name=='beta':
        plot_data = []
        for row in range(6):
            weights =  exp_and_normalise(particles.weights)
            hist, bin_edges = np.histogram(ps_smc2[name][:,row,0], weights = weights)
            center_bin_edges = bin_edges[1:] + ((bin_edges[1:] - bin_edges[:-1])/2.)

            plot_data_row = pd.DataFrame(hist, columns=['value'])
            plot_data_row['bin'] = center_bin_edges
            plot_data_row['row'] = row
            plot_data_row['col'] = 1
            plot_data.append(plot_data_row)
        return pd.concat(plot_data)
    if name=='betabeta':
        plot_data = []
        for row in range(6):
            for col in range(6):
                weights =  exp_and_normalise(particles.weights)
                hist, bin_edges = np.histogram(ps_smc2[name][:,row,col], weights = weights)
                center_bin_edges = bin_edges[1:] + ((bin_edges[1:] - bin_edges[:-1])/2.)

                plot_data_row = pd.DataFrame(hist, columns=['value'])
                plot_data_row['bin'] = center_bin_edges
                plot_data_row['row'] = row
                plot_data_row['col'] = col
                plot_data.append(plot_data_row)
        return pd.concat(plot_data)
    

def plot_weighted_histogram(df, width=300, height=50) :
    # it only works for one chain for now
    c = alt.Chart(df).mark_bar(
        width = 10,
        ).encode(
        alt.X('bin:Q', title=None),
        alt.Y('value:Q'),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
        ).resolve_scale(
            x='independent'
        ).properties(width=width, height=height)
    return c


In [29]:
# ps_smc2 = post_process_sign(ps_smc2)
pdata = get_histogram_data(ps_smc2, 'alpha')
pdata['source'] = 'smc2'

plot_weighted_histogram(pdata, height=150, width = 300)

alt.Chart(...)

In [37]:
pdata = get_histogram_data(ps_smc2, 'beta')
pdata['source'] = 'smc2'

pdata

plot_weighted_histogram(pdata)

alt.Chart(...)

In [31]:
log_dir3 = 'log/20210222_124703_smc2test/'
exp_data = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
# particles.particles[5].particles['beta']

In [32]:
particles = load_obj('particles', log_dir3)
particles.resample_particles_bundles()
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
# ps_smc2['betabeta'] = get_betabeta(ps_smc2['beta'])

##

## Or load existing directory

In [33]:
# log_dir = '/log/20210122_173348_mcmc_ibis_adapt9/'
# ps = load_obj( 'mcmc_post_samples', log_dir)
# ps['beta'] = ps['beta'][num_warmup:].copy()

## Post process loadings for sign flips

In [34]:
# ps_hmc = post_process_sign(ps_hmc)
# ps_mcmc = post_process_sign(ps_mcmc)
# ps_smc2 = post_process_sign(ps_smc2)

# ps_mcmc['betabeta'] = get_betabeta(ps_mcmc['beta'])
# ps_hmc['betabeta'] = get_betabeta(ps_hmc['beta'])

In [41]:
df

,idx,row,col,value,source
0,0,0,0,0.464272,mcmc
1,1,0,0,0.572665,mcmc
2,2,0,0,0.635054,mcmc
3,3,0,0,0.451931,mcmc
4,4,0,0,0.615930,mcmc
...,...,...,...,...,...
5995,995,5,0,0.928057,mcmc
5996,996,5,0,0.900724,mcmc
5997,997,5,0,0.922630,mcmc
5998,998,5,0,1.315003,mcmc


## Plot MCMC samples

In [42]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'

df2 = get_post_df(ps_hmc[param]) 
df2['source'] = 'hmc'

df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_histogram(pd.concat([df,df2, df3]), opacity = 0.7, height=100)

alt.Chart(...)

In [154]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'

df2 = get_post_df(ps_hmc[param]) 
df2['source'] = 'hmc'


df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_histogram(pd.concat([df,df2, df3]), height=100)


alt.VConcatChart(...)

## Plot quantiles

In [21]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [22]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [23]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)